# Build the multi-modal for intermodal route planning

**Nodes**: Stations in the SBB traffic network. Curated from the following
sources:

- `regional_station.csv` from
  [here](https://data.sbb.ch/explore/dataset/regionale-fahrplane/information/)

- `city_station.csv` from
  [here](https://data.sbb.ch/explore/dataset/stadtefahrplan/information/)

- `pr_station.csv` from
  [here](https://data.sbb.ch/explore/dataset/mobilitat/information/)

Each node has the `station_name` as the primary attribute and then also
information about the location of the station (`pos` (tuple of lon, lat float),
`loc` (string repr of the former), `opucid` (an identifier) and `abbr` (the
abbreviation of the station name)).

**Edges**: Trip segments between stations for multiple modalities (foot, bike,
car, train). The latter sourced from the timetable of the SBB for a given day
(see `timetable.ipynb` for details). The former are generated from the `nodes`
by computing all pairs of nodes that are within a given time travel threshold
(e.g. all stations within 30 minutes foot distance).

Three types of nodes:

- Static for modalities `foot`, `bike` and `car` from each node given a time
  threshold
- Dynamic given start position for all modalities given with k-nearest neighbors
- ...

Results: Directed, multi-edged graph


In [ ]:
# Immports
import pandas as pd
import networkx as nx

from tqdm import tqdm
import pickle

In [ ]:
# Load the data
df = pd.read_csv("../data/ist-daten-sbb.csv", sep=";")

# Select and rename columns
cols = {
    "Journey identifier": "journey_id",
    "Arrival time": "arrival",
    "Departure time": "departure",
    "Stop name": "station",
    "OPUIC": "opuic",
    "Geopos": "pos",
}

# Select and rename columns
df = df[cols.keys()].rename(columns=cols)

# Convert time columns to datetime
df["arrival"] = pd.to_datetime(df["arrival"])
df["departure"] = pd.to_datetime(df["departure"])

# Show the result
df.head(5)

## Add Route Network from SBB Timetable

Using data from `01.12.2023`


In [ ]:
# Build edge list (with edge attributes) for train journeys
edges = []
for journey_id in tqdm(df.journey_id.unique()):
    trip = df[df.journey_id == journey_id].sort_values("departure", inplace=False)
    trip_name = f"{trip.iloc[0].station} -> {trip.iloc[-1].station}"

    for i in range(len(trip) - 1):
        edges.append(
            (
                trip.iloc[i].station,
                trip.iloc[i + 1].station,
                {
                    "departure": trip.iloc[i].departure,
                    "arrival": trip.iloc[i + 1].arrival,
                    "duration": trip.iloc[i + 1].arrival - trip.iloc[i].departure,
                    "journey_id": journey_id,
                    "trip_name": trip_name,
                    "type": "train",
                },
            )
        )

In [ ]:
# Initialise multi-graph object
G = nx.MultiDiGraph(edges)

# We have the following attributes for each edge (for train journeys)
list(G.edges(data=True))[0]

In [ ]:
# Add node attributes (station id and position)
for node in tqdm(G.nodes):
    # Get the station attributes from the dataframe
    station = df[df.station == node].iloc[0]
    station_attributes = {"pos": station.pos, "opuic": station.opuic}

    # Add the attributes to the node
    G.nodes[node].update(station_attributes)

Add information about whether station has parking spot (`has_pr`)


In [ ]:
# Read in city and regional stations
pr_stations = pd.read_csv("../data/pr_stations.csv")

# Rename columns
pr_stations = pr_stations.rename(columns={"station": "name", "station_abbr": "abbr"})

# Initialise has_pr to False for all nodes
for node in G.nodes:
    G.nodes[node]["has_pr"] = False

not_found = 0
for pr_station in pr_stations.name.unique():
    if pr_station in G.nodes:
        G.nodes[pr_station]["has_pr"] = True
    else:
        not_found += 1

print(
    f"Added {pr_stations.name.nunique() - not_found} of {pr_stations.name.nunique()} PR stations. {not_found} not found."
)

In [ ]:
# We have the following attributes for each node
list(G.nodes(data=True))[0]

## Visualise graph


In [ ]:
# Set up the map projection and the transformation
proj = ccrs.Mercator()
transform = ccrs.Geodetic()

# Create a figure with an axes set with the projection
fig, ax = plt.subplots(subplot_kw={"projection": proj}, figsize=(30, 10))

# Set the extent of the map (min longitude, max longitude, min latitude, max latitude)
ax.set_extent([5, 12, 45.5, 48], crs=ccrs.PlateCarree())

# Add map features
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=":")

# Draw nodes (use scatter for individual node plotting)
for _, attr in G.nodes(data=True):
    lon, lat = attr["pos"]
    col = "red" if attr["has_pr"] else "blue"
    ax.scatter(lat, lon, s=10, color=col, transform=transform, zorder=3)

# Draw edges (use scatter for individual edge plotting)
# for edge in G.edges():
#    lons, lats = zip(*[pos[node] for node in edge])
#    ax.plot(lons, lats, color='gray', linewidth=2, transform=transform, zorder=2)

# Add node labels
# for node, (lon, lat) in pos.items():
#   plt.text(lon-0.02, lat-0.015, node, transform=transform, horizontalalignment='right')

print("All city and regional stations")
plt.show()

### Save the graph


In [ ]:
with open("../data/graph.pickle", "wb") as file:
    pickle.dump(G, file)

In [ ]:
len([node for node in G.nodes() if G.nodes[node]["has_pr"]])